# reproduce GCN

In [2]:
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
!pip install torch_geometric

In [5]:
import torch
import numpy as np
import scipy.sparse as sp
from torch_geometric.datasets import Planetoid

In [34]:
dataset_name = "CiteSeer"
dataset = Planetoid(root='/tmp/dataset', name=dataset_name)
data = dataset[0]

Processing...
Done!


In [35]:
print(data)
edge_index = data.edge_index.numpy()
features = data.x.numpy()
labels = data.y.numpy()
train_mask = data.train_mask.numpy()
val_mask = data.val_mask.numpy()
test_mask = data.test_mask.numpy()


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [46]:
print(edge_index.shape)
print(features.shape)
print(labels.shape)
print(train_mask.shape)
print(val_mask.shape)
print(test_mask.shape)
print(edge_index)

(2, 9104)
(3327, 3703)
(3327,)
(3327,)
(3327,)
(3327,)
[[ 628  158  486 ... 2820 1643   33]
 [   0    1    1 ... 3324 3325 3326]]


In [47]:
class GraphData:
    def __init__(self, edge_index, num_nodes, features, labels, train_mask, val_mask, test_mask):
        self.num_nodes = num_nodes
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

        self.adj = self.build_adjacency_matrix(edge_index, num_nodes)
        self.norm_adj = self.normalize_adjacency(self.adj)

        self.train_mask = torch.tensor(train_mask, dtype=torch.bool)
        self.val_mask = torch.tensor(val_mask, dtype=torch.bool)
        self.test_mask = torch.tensor(test_mask, dtype=torch.bool)

    def build_adjacency_matrix(self, edge_index, num_nodes):
        row, col = edge_index
        values = np.ones(len(row))
        adj = sp.coo_matrix((values, (row, col)), shape=(num_nodes, num_nodes), dtype=np.float32)
        adj = adj + sp.eye(num_nodes)
        return adj

    def normalize_adjacency(self, adj):
        degree = np.array(adj.sum(axis=1)).flatten()
        degree_inv_sqrt = np.power(degree, -0.5)
        degree_inv_sqrt[np.isinf(degree_inv_sqrt)] = 0
        D_inv_sqrt = sp.diags(degree_inv_sqrt)

        norm_adj = D_inv_sqrt @ adj @ D_inv_sqrt
        return torch.tensor(norm_adj.toarray(), dtype=torch.float32)

In [48]:
graph_data = GraphData(edge_index, data.num_nodes, features, labels, train_mask, val_mask, test_mask)

In [52]:
def compute_laplacian_loss(output, adj):

    degree = torch.diag(adj.sum(1))
    laplacian = degree - adj
    lap_loss = torch.trace(output.T @ laplacian @ output)
    return lap_loss

In [53]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.randn(in_features, out_features) * 0.01)

    def forward(self, A_norm, H):
        return torch.matmul(A_norm, torch.matmul(H, self.weight))

In [58]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(input_dim, hidden_dim)
        self.gcn2 = GCNLayer(hidden_dim, output_dim)

    def forward(self, A_norm, X):
        H = self.gcn1(A_norm, X)
        H = F.relu(H)
        H = self.gcn2(A_norm, H)
        return F.softmax(H, dim=1)

In [68]:
import torch.optim as optim

def train(model, optimizer, features, adj, labels, train_mask, lambda_reg=5e-4, epochs=200):
    model.train()
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(adj, features)
        loss_sup = loss_fn(output[train_mask], labels[train_mask])
        loss_reg = compute_laplacian_loss(output, adj)
        loss = loss_sup + lambda_reg * loss_reg
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Loss = {loss.item():.4f}")


def evaluate(model, features, adj, labels, mask):
    model.eval()
    with torch.no_grad():
        output = model(adj, features)
        pred = output.argmax(dim=1)
        acc = (pred[mask] == labels[mask]).sum().item() / mask.sum().item()
    return acc

input_dim = graph_data.features.shape[1]
hidden_dim = 16
output_dim = len(set(graph_data.labels.numpy()))

gcn = GCN(input_dim, hidden_dim, output_dim)
optimizer = optim.Adam(gcn.parameters(), lr=0.01, weight_decay=5e-4)

train(gcn, optimizer, graph_data.features, graph_data.norm_adj, graph_data.labels, graph_data.train_mask, epochs=200)

train_acc = evaluate(gcn, graph_data.features, graph_data.norm_adj, graph_data.labels, graph_data.train_mask)
val_acc = evaluate(gcn, graph_data.features, graph_data.norm_adj, graph_data.labels, graph_data.val_mask)
test_acc = evaluate(gcn, graph_data.features, graph_data.norm_adj, graph_data.labels, graph_data.test_mask)


Epoch 0: Loss = 1.7918
Epoch 10: Loss = 1.6147
Epoch 20: Loss = 1.2058
Epoch 30: Loss = 1.1063
Epoch 40: Loss = 1.0944
Epoch 50: Loss = 1.0918
Epoch 60: Loss = 1.0859
Epoch 70: Loss = 1.0830
Epoch 80: Loss = 1.0808
Epoch 90: Loss = 1.0790
Epoch 100: Loss = 1.0776
Epoch 110: Loss = 1.0764
Epoch 120: Loss = 1.0753
Epoch 130: Loss = 1.0744
Epoch 140: Loss = 1.0736
Epoch 150: Loss = 1.0728
Epoch 160: Loss = 1.0722
Epoch 170: Loss = 1.0717
Epoch 180: Loss = 1.0712
Epoch 190: Loss = 1.0707


In [69]:
# import itertools
# param_grid = {
#     "hidden_dim": [16, 32, 64],
#     "lr": [0.01, 0.005, 0.001],
#     "weight_decay": [5e-4, 1e-3, 5e-3]
# }


# param_combinations = list(itertools.product(*param_grid.values()))


# best_params = None
# best_val_acc = 0.0


# for params in param_combinations:
#     hidden_dim, lr, weight_decay = params
#     print(f"\n🔹 model - hidden_dim={hidden_dim}, lr={lr}, weight_decay={weight_decay}")

#     model = GCN(input_dim, hidden_dim, output_dim)
#     optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
#     loss_fn = torch.nn.CrossEntropyLoss()

#     train(model, optimizer, graph_data.features, graph_data.norm_adj, graph_data.labels, graph_data.train_mask, epochs=200)

#     val_acc = evaluate(model, graph_data.features, graph_data.norm_adj, graph_data.labels, graph_data.val_mask)
#     print(f"Validation Accuracy: {val_acc:.4f}")

#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         best_params = params

# print(f"Valid: Hidden Dim: {best_params[0]}, Learning Rate: {best_params[1]}, Weight Decay: {best_params[2]}")

# best_hidden_dim, best_lr, best_weight_decay = best_params
# best_model = GCN(input_dim, best_hidden_dim, output_dim)
# best_optimizer = optim.Adam(best_model.parameters(), lr=best_lr, weight_decay=best_weight_decay)

# train(best_model, best_optimizer, graph_data.features, graph_data.norm_adj, graph_data.labels, graph_data.train_mask, epochs=200)

# test_acc = evaluate(best_model, graph_data.features, graph_data.norm_adj, graph_data.labels, graph_data.test_mask)
# print(f"{test_acc:.4f}")

In [70]:
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

Train Accuracy: 1.0000
Validation Accuracy: 0.6760
Test Accuracy: 0.6990


In [73]:
dataset_cora = Planetoid(root='/tmp/Cora', name="Cora")
data_cora = dataset_cora[0]

input_dim_cora = data_cora.x.shape[1]
output_dim_cora = len(set(data_cora.y.numpy()))

graph_data_cora = GraphData(
    data_cora.edge_index.numpy(), data_cora.num_nodes,
    data_cora.x.numpy(), data_cora.y.numpy(),
    data_cora.train_mask.numpy(), data_cora.val_mask.numpy(), data_cora.test_mask.numpy()
)

model_cora = GCN(input_dim_cora, hidden_dim, output_dim_cora)
optimizer_cora = optim.Adam(model_cora.parameters(), lr=0.01, weight_decay=5e-4)

train(model_cora, optimizer_cora, graph_data_cora.features, graph_data_cora.norm_adj, graph_data_cora.labels, graph_data_cora.train_mask, epochs=200)

test_acc_cora = evaluate(model_cora, graph_data_cora.features, graph_data_cora.norm_adj, graph_data_cora.labels, graph_data_cora.test_mask)
print(f"Test Accuracy on Cora: {test_acc_cora:.4f}")


Epoch 0: Loss = 1.9459
Epoch 10: Loss = 1.8724
Epoch 20: Loss = 1.5616
Epoch 30: Loss = 1.3011
Epoch 40: Loss = 1.2359
Epoch 50: Loss = 1.2300
Epoch 60: Loss = 1.2294
Epoch 70: Loss = 1.2253
Epoch 80: Loss = 1.2225
Epoch 90: Loss = 1.2208
Epoch 100: Loss = 1.2193
Epoch 110: Loss = 1.2181
Epoch 120: Loss = 1.2170
Epoch 130: Loss = 1.2160
Epoch 140: Loss = 1.2152
Epoch 150: Loss = 1.2144
Epoch 160: Loss = 1.2138
Epoch 170: Loss = 1.2132
Epoch 180: Loss = 1.2126
Epoch 190: Loss = 1.2121
Test Accuracy on Cora: 0.8250


In [ ]:
dataset_pubmed = Planetoid(root='/tmp/PubMed', name="PubMed")
data_pubmed = dataset_pubmed[0]

input_dim_pubmed = data_pubmed.x.shape[1]
output_dim_pubmed = len(set(data_pubmed.y.numpy()))

graph_data_pubmed = GraphData(
    data_pubmed.edge_index.numpy(), data_pubmed.num_nodes,
    data_pubmed.x.numpy(), data_pubmed.y.numpy(),
    data_pubmed.train_mask.numpy(), data_pubmed.val_mask.numpy(), data_pubmed.test_mask.numpy()
)

model_pubmed = GCN(input_dim_pubmed, 16, output_dim_pubmed)
optimizer_pubmed = optim.Adam(model_pubmed.parameters(), lr=0.01, weight_decay=5e-4)

train(model_pubmed, optimizer_pubmed, graph_data_pubmed.features, graph_data_pubmed.norm_adj, graph_data_pubmed.labels, graph_data_pubmed.train_mask, epochs=200)

test_acc_pubmed = evaluate(model_pubmed, graph_data_pubmed.features, graph_data_pubmed.norm_adj, graph_data_pubmed.labels, graph_data_pubmed.test_mask)
print(f"Test Accuracy on PubMed: {test_acc_pubmed:.4f}")

Epoch 0: Loss = 1.0986
Epoch 10: Loss = 1.0870
Epoch 20: Loss = 1.0440
Epoch 30: Loss = 0.9871
Epoch 40: Loss = 0.9333
Epoch 50: Loss = 0.8902
Epoch 60: Loss = 0.8589
Epoch 70: Loss = 0.8367
Epoch 80: Loss = 0.8184
